In [1]:
import os
import pymysql
import pandas as pd
password = os.environ.get('MYSQL_PASSWORD')


In [2]:
password = os.environ.get('MYSQL_PASSWORD')

try:
    con = pymysql.connect(
    host='localhost',
    user='root',
    password=password,
    charset='utf8mb4'
    )
    print("Connection to the database was successful!")
except pymysql.Error as e:
    print(f"An error occurred while connecting to the database: {e}")


Connection to the database was successful!


In [3]:
def connector (database, password=password, host='localhost', 
               user='root', charset='utf8mb4',):
    
    conn = pymysql.connect(
                        host=host,
                        user=user,
                        password=password,
                        charset=charset, 
                        database=database)
    
    return conn

In [4]:
con = connector ('sql_inventory')
df = pd.read_sql("SELECT * FROM products", con)
df


,product_id,name,quantity_in_stock,unit_price
0,1,Foam Dinner Plate,70,1.21
1,2,"Pork - Bacon,back Peameal",49,4.65
2,3,"Lettuce - Romaine, Heart",38,3.35
3,4,"Brocolinni - Gaylan, Chinese",90,4.53
4,5,Sauce - Ranch Dressing,94,1.63
5,6,Petit Baguette,14,2.39
6,7,Sweet Pea Sprouts,98,3.29
7,8,Island Oasis - Raspberry,26,0.74
8,9,Longan,67,2.26
9,10,Broom - Push,6,1.09
